In [10]:
import sys
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

sys.path.append('../src/')
from utils import merge_lat_lng

## This file is for further process about feature generated by GPT for re-ranking

In [11]:
feature = pd.read_csv('../data/station_personalized_features.csv', index_col=0)

In [12]:
user = pd.read_csv('../data/user_profile.csv', index_col=0)

In [13]:
feature.head()

,NAME__positive,accessibility_and_availability_positive,amenities_and_location_positive,appearance_positive,availability_and_accessibility_positive,charging_experience_positive,charging_speed_and_efficiency_positive,cleanliness_positive,comfort_positive,comfort_and_convenience_positive,...,price_and_cost_negative,privacy_and_tracking_negative,queue_and_waiting_time_negative,reliability_and_maintenance_negative,safety_negative,sentiment_negative,sustainability_negative,time_negative,time_limit_negative,user_interface_and_mobile_app_negative
docid,,,,,,,,,,,,,,,,,,,,,
33921,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39114,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39713,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
user.head()

,NAME__positive,accessibility_and_availability_positive,amenities_and_location_positive,appearance_positive,availability_and_accessibility_positive,charging_experience_positive,charging_speed_and_efficiency_positive,cleanliness_positive,comfort_positive,comfort_and_convenience_positive,...,price_and_cost_negative,privacy_and_tracking_negative,queue_and_waiting_time_negative,reliability_and_maintenance_negative,safety_negative,sentiment_negative,sustainability_negative,time_negative,time_limit_negative,user_interface_and_mobile_app_negative
0,0,5,3,0,5,2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,3,0,5,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
np.save('../data/encoded_station.npy', feature.to_numpy())

In [16]:
row_to_docid = feature.index.to_list()
file_path = '../data/row_to_docid.txt'

# Open the file in write mode
with open(file_path, 'w') as file:
    # Write each element of the list as a new line
    for element in row_to_docid:
        file.write(str(element) + '\n')

In [8]:
np.save('../data/encoded_user_profile.npy', user.to_numpy())

## Vector Ranker
used for re-rank

In [12]:
encoded_docs = np.load('../data/encoded_station.npy')
user_profile = np.load('../data/encoded_user_profile.npy')

file_path = '../data/row_to_docid.txt'
with open(file_path, 'r') as file:
    row_to_docid = file.read().splitlines()

row_to_docid = [int(i) for i in row_to_docid]


profile_row_to_userid = [1, 2]

In [13]:
from vector_ranker import VectorRanker
Vec_Ranker = VectorRanker(bi_encoder_model_name=None, 
                          encoded_docs=encoded_docs, 
                          row_to_docid=row_to_docid, 
                          user_profile=user_profile, 
                          profile_row_to_userid=profile_row_to_userid)


In [14]:
# pesudo-result testing
result = row_to_docid[:100]

In [15]:
Vec_Ranker.personalized_re_rank(result, user_id=1)[:10] 

[1711, 443, 324, 337, 347, 350, 367, 383, 397, 401]

In [16]:
Vec_Ranker.personalized_re_rank(result, user_id=2)[:10]

[350, 985, 1178, 347, 306, 311, 236, 221, 803, 936]